In [10]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhanghanyi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
X_train = open('X_train.txt')
line = X_train.readline()
train = list()
while line:
    line = line.strip('\n')
    line = line.lower()
    line = line.split()
#     print(line)
    temp_line = list()
    for word in line: 
        if word not in en_stops:
            temp_line.append(word)
    train.append(temp_line)
#     print(temp_line)
    line = X_train.readline()
print(type(train))
print(len(train))
print(type(train[0]))

<class 'list'>
341072
<class 'list'>


In [18]:
from keras.layers.core import Activation, Dense
from keras.preprocessing.text import Tokenizer
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing import sequence
import nltk 
import collections
import numpy as np

Using TensorFlow backend.
/Users/zhanghanyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/zhanghanyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/zhanghanyi/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/zhanghanyi/anaconda3/lib/python3.6

In [12]:
y_train = open('y_train.txt')
label = list()
line = y_train.readline()
while line:
    line = line.strip('\n')
    line = line.split()
    label.append(int(line[0]))
    line = y_train.readline()
print(len(label))

341072


In [13]:
X_valid = open('X_valid.txt')
line = X_valid.readline()
valid_data = list()
while line:
    line = line.strip('\n')
    line = line.lower()
    line = line.split()
#     print(line)
    temp_line = list()
    for word in line: 
        if word not in en_stops:
            temp_line.append(word)
    valid_data.append(temp_line)
#     print(temp_line)
    line = X_valid.readline()
print(type(valid_data))
print(len(valid_data))
print(type(valid_data[0]))

<class 'list'>
113691
<class 'list'>


In [15]:
y_valid = open('y_valid.txt')
valid_label = list()
line = y_valid.readline()
while line:
    line = line.strip('\n')
    line = line.split()
    valid_label.append(int(line[0]))
    line = y_valid.readline()
print(len(valid_label))

113691


In [16]:
import numpy as np
label = np.array(label)
train = np.array(train)
valid_data = np.array(valid_data)
valid_label = np.array(valid_label)

In [19]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train)
tokenizer.fit_on_texts(valid_data)

In [20]:
vocab=tokenizer.word_index

In [21]:
len(vocab)

448227

In [22]:
encoded_train = tokenizer.texts_to_sequences(train)
encoded_valid = tokenizer.texts_to_sequences(valid_data)

In [23]:
type(encoded_valid)

list

In [24]:
encoded_train = np.array(encoded_train)
encoded_valid = np.array(encoded_valid)

In [25]:
temp_len = 0
for i in range(len(encoded_train)):
    temp_len += len(encoded_train[i])

mean_len = temp_len / len(encoded_train)
mean_len = int(mean_len)

encoded_train = sequence.pad_sequences(encoded_train, maxlen=mean_len)

In [26]:
encoded_valid = sequence.pad_sequences(encoded_valid, maxlen=mean_len)

In [27]:
label = label.reshape(-1,1)
valid_label = valid_label.reshape(-1,1)

In [28]:
valid_label.shape

(113691, 1)

In [29]:
import keras
label_train = keras.utils.to_categorical(label)
valid_label = keras.utils.to_categorical(valid_label)

In [30]:
len(vocab)

448227

In [31]:
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras import layers
from keras.optimizers import Adam
from keras.models import Model,Sequential
import keras
embedding_dim = 128
model = Sequential()
model.add(layers.Embedding(len(vocab)+1, embedding_dim,input_length=mean_len))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(Dense(6,activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 43, 128)           57373184  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 39, 128)           82048     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 774       
Total params: 57,456,006
Trainable params: 57,456,006
Non-trainable params: 0
_________________________________________________________________
None


In [32]:
from keras.callbacks import ModelCheckpoint

filepath = "best_weights.h5"
 
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max', period=1)
callbacks_list = [checkpoint]

adam = keras.optimizers.Adamax(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(encoded_train, label_train,epochs=2,verbose=1,validation_data = (encoded_valid,valid_label),batch_size=500,callbacks = callbacks_list)

Train on 341072 samples, validate on 113691 samples
Epoch 1/2
 58500/341072 [====>.........................] - ETA: 7:03 - loss: 1.2129 - acc: 0.6272

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Valid'], loc='upper left')
plt.show()

In [120]:
X_test = open('X_test.txt')
line = X_test.readline()
test = list()
while line:
    line = line.strip('\n')
    line = line.lower()
    line = line.split()
    temp_line = list()
    for word in line: 
        if word not in en_stops:
            temp_line.append(word)
    test.append(temp_line)
    line = X_test.readline()
print(type(test))
print(len(test))
print(type(test[0]))

<class 'list'>
113691
<class 'list'>


In [138]:
y_test = open('y_test.txt')
label_test = list()
line = y_test.readline()
while line:
    line = line.strip('\n')
    line = line.split()
    label_test.append(int(line[0]))
    line = y_test.readline()

In [122]:
encoded_test = tokenizer.texts_to_sequences(test)
encoded_test = np.array(encoded_test)
encoded_test = sequence.pad_sequences(encoded_test, maxlen=mean_len)

In [141]:
from keras.models import load_model

model = load_model('best_weights.h5')

predict_test = model.predict(encoded_test)

In [142]:
acc_num = 0
for i in range(len(encoded_test)):
    cla = np.argmax(predict_test[i])
    if cla == label_test[i]:
        acc_num = acc_num + 1

print(acc_num/len(encoded_test))

0.7049810451135095
